## 구글 안드로이드 앱 스토어 분석 
- Plotly를 통해, 안드로이드 앱스토어 데이터의 데이터 시각화를 고급지게 꾸며보자! 

<img src="data/app_store.png">

- 자기만의 iOS나 안드로이드 앱을 만드는 것을 생각해본적이 있는가? 만약 그렇다면 앱스토어에서 이것들이 어떻게 작동하는지에 대해서도 궁금해 했을 것이다.
- 이번 시간에는 "**Annie**" 나 "**Sensor Tower**"와 같은 회사에서 제공하는 앱스토어 분석 중 일부를 가져와, 많은 회사에서 사용하는 개발이나 앱 마케팅 전략에 대해 살펴보고자 한다. 
- 이번 시간에서는, 구글 플레이 스토어의 수천 개의 앱을 비교하여 아래와 같은 통찰을 얻을 수 있다.
    - 다양한 앱 범주(게임, 라이프스타일, 날씨 등)가 얼마나 경쟁력이 있는가?
    - 어떤 앱 범주가 인기에 따라 흥미있는 기회를 제공하는가?
    - 무료 앱에 비해 유료앱은 얼마나 많은 다운로드 횟수를 포기해야 하는가?
    - 유료 앱에 합리적인 금액은 얼마인가?
    - 가장 높은 매출을 올린 유로앱은 무엇인가?
    - 판매 수익에 기반하여, 개발 비용을 회수하는 유료 앱의 수는 얼마나 되는가? 

- 또한 위의 통찰을 얻음과 동시에 아래의 사항을 배울 수 있다. 
    - 중복 항목을 빠르게 제거하는 방법 
    - 원하지 않는 기호를 제거하고, 데이터를 숫자 형식으로 변경하는 방법 
    - Pandas를 통해, 중첩된 데이터가 있는 열을 처리하는 방법 
    - Plotly 라이브러리로 매력적인 데이터 시각화 만드는 방법
    - 수직, 수평, 그룹화된 막대 차트 생성
    - 범주형 데이터 대한 파이 차트, 도넛 차트 생성 
    - 색상 눈금을 이용해 보기 좋은 산점도 생성

### 1. 데이터 정제: NaN값과 중복값 제거 

In [1]:
# 필요한 모듈 임포트 
import pandas as pd 
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

#### 1-1. 데이터 살펴보기 

In [6]:
# app.csv 파일 불러오기 및 데이터 출력 
df_app_store = pd.read_csv('data/apps.csv')
print(df_app_store.shape)
df_app_store.head()

(10841, 12)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
0,Ak Parti Yardım Toplama,SOCIAL,NaN,0,8.7,0,Paid,$13.99,Teen,Social,"July 28, 2017",4.1 and up
1,Ain Arabic Kids Alif Ba ta,FAMILY,NaN,0,33.0,0,Paid,$2.99,Everyone,Education,"April 15, 2016",3.0 and up
2,Popsicle Launcher for Android P 9.0 launcher,PERSONALIZATION,NaN,0,5.5,0,Paid,$1.49,Everyone,Personalization,"July 11, 2018",4.2 and up
3,Command & Conquer: Rivals,FAMILY,NaN,0,19.0,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device
4,CX Network,BUSINESS,NaN,0,10.0,0,Free,0,Everyone,Business,"August 6, 2018",4.1 and up


- "apps.csv"의 데이터 수는 10,841개의 행과 12개의 컬럼으로 구성되어 있다. 
- 컬럼의 의미는 아래와 같다. 
    - App: App Store에 있는 App 이름 
    - Category: 해당 App의 카테고리 
    - Rating: 해당 App의 평점 (5.0점이 만점)
    - Reviews: 해당 App의 리뷰 개수 
    - Size_MBs: 해당 App의 설치 용량 (단위: MBs)
    - Installs: 해당 App의 다운로드, 설치 수
    - Type: 해당 App의 타입 - 유료앱: Paid / 무료앱: Free
    - Price: 해당 App의 가격 (단위 `$`) - 유료앱: 가격 있음 / 무료앱: 0
    - Content_Rating: 해당 App의 이용 등급 
    - Genres: 해당 App의 장르
    - Last_Updated: 해당 App의 마지막 업데이트 일시 
    - Android_Ver: 해당 App 설치시 안드로이드 버전 조건
- 여기서 우리는 몇 가지의 데이터 전처리가 필요하다는 것을 확인할 수 있다. 
    - 필요없는 컬럼 제거
    - NaN 값 해결 - Rating, Type
    - Price 컬럼에 달러 표시가 된 것으로 보아 str일 것으로 예상 해당 컬럼 int로 변환

In [9]:
# 임의의 5개 표본 데이터 확인
df_app_store.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
5289,AE GTO Racing,GAME,3.9,6988,35.0,"100,000",Free,0,Everyone 10+,Racing,"September 24, 2015",Varies with device
7820,Do Not Crash,FAMILY,3.7,56664,8.3,"1,000,000",Free,0,Everyone,Casual,"February 5, 2015",2.3 and up
1579,BJ Toys,SHOPPING,4.7,55,8.0,500,Free,0,Everyone,Shopping,"April 11, 2018",4.0.3 and up
979,NOMISMA.com.cy by FMW,NEWS_AND_MAGAZINES,5.0,3,6.3,100,Free,0,Everyone,News & Magazines,"May 29, 2016",4.0 and up
4432,Graffiti Letters (A-Z),LIFESTYLE,3.9,386,7.9,"50,000",Free,0,Everyone,Lifestyle,"February 18, 2017",4.0 and up


- ".sample(n)"은 임의의 n개 행을 반환 한다. 이것은 우리가 데이터프레임을 검사하는 편리한 방법 중 하나이다.

#### 1-2. 데이터 전처리 1 - 불필요한 컬럼 및 NaN값 제거

In [10]:
# 필요없는 Last_Updated와 Android_Ver 컬럼 삭제 
df_app_store = df_app_store.drop(['Last_Updated', 'Android_Ver'], axis=1)
df_app_store.head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
0,Ak Parti Yardım Toplama,SOCIAL,NaN,0,8.7,0,Paid,$13.99,Teen,Social
1,Ain Arabic Kids Alif Ba ta,FAMILY,NaN,0,33.0,0,Paid,$2.99,Everyone,Education
2,Popsicle Launcher for Android P 9.0 launcher,PERSONALIZATION,NaN,0,5.5,0,Paid,$1.49,Everyone,Personalization
3,Command & Conquer: Rivals,FAMILY,NaN,0,19.0,0,NaN,0,Everyone 10+,Strategy
4,CX Network,BUSINESS,NaN,0,10.0,0,Free,0,Everyone,Business


- 먼저 원하지 않는 컬럼을 제거하려면 ".drop()" 메소드에 제거하려는 컬럼 이름을 전달하면 된다. (여기서는 'Last_Updated'와 'Android_Ver'이다.)
- 또한 "axis=1"을 설정해서 특정 열을 삭제하도록 하면 된다.

In [12]:
# 사용하지 않는 NaN값 제거 - NaN 값 확인
df_app_store.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size_MBs             0
Installs             0
Type                 1
Price                0
Content_Rating       0
Genres               0
dtype: int64

In [15]:
# Rating 컬럼 기준 NaN 값 확인
nan_row = df_app_store[df_app_store.Rating.isna()]
print(nan_row.shape)
nan_row.head()

(1474, 10)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
0,Ak Parti Yardım Toplama,SOCIAL,NaN,0,8.7,0,Paid,$13.99,Teen,Social
1,Ain Arabic Kids Alif Ba ta,FAMILY,NaN,0,33.0,0,Paid,$2.99,Everyone,Education
2,Popsicle Launcher for Android P 9.0 launcher,PERSONALIZATION,NaN,0,5.5,0,Paid,$1.49,Everyone,Personalization
3,Command & Conquer: Rivals,FAMILY,NaN,0,19.0,0,NaN,0,Everyone 10+,Strategy
4,CX Network,BUSINESS,NaN,0,10.0,0,Free,0,Everyone,Business


In [16]:
# NaN 값 제거 
df_apps_clean = df_app_store.dropna()
df_apps_clean

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.0,4,25.0,1,Free,0,Everyone,Medical
28,Ra Ga Ba,GAME,5.0,2,20.0,1,Paid,$1.49,Everyone,Arcade
47,Mu.F.O.,GAME,5.0,2,16.0,1,Paid,$0.99,Everyone,Arcade
82,Brick Breaker BR,GAME,5.0,7,19.0,5,Free,0,Everyone,Arcade
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.0,1,4.6,5,Free,0,Everyone,Medical
...,...,...,...,...,...,...,...,...,...,...
10836,Subway Surfers,GAME,4.5,27723193,76.0,"1,000,000,000",Free,0,Everyone 10+,Arcade
10837,Subway Surfers,GAME,4.5,27724094,76.0,"1,000,000,000",Free,0,Everyone 10+,Arcade
10838,Subway Surfers,GAME,4.5,27725352,76.0,"1,000,000,000",Free,0,Everyone 10+,Arcade
10839,Subway Surfers,GAME,4.5,27725352,76.0,"1,000,000,000",Free,0,Everyone 10+,Arcade


- NaN 값이 있는 행을 찾아서 제거하기 위해 ".isna()"의 값이 "True"인 위치에 대한 데이터프레임의 하위 집합을 만들 수 있다.
- Rating 열의 NaN 값이 리뷰가 없는 곳과 연관되어 있는 것을 알 수 있다.
- 찾은 NaN 값이 불필요한 데이터 이므로 우선 NaN 값을 제거하였다. 삭제후 데이터의 수는 "9,367개의 행"과 "10개의 컬럼으로 바뀌었다.

#### 1-2. 중복된 데이터 제거 

In [22]:
# 중복 데이터 개수 확인 
df_apps_clean.duplicated().sum()

476

In [23]:
# 중복된 항목 출력
duplicated_rows = df_apps_clean[df_apps_clean.duplicated]
print(duplicated_rows.shape)
duplicated_rows.head()

(476, 10)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
946,420 BZ Budeze Delivery,MEDICAL,5.0,2,11.0,100,Free,0,Mature 17+,Medical
1133,MouseMingle,DATING,2.7,3,3.9,100,Free,0,Mature 17+,Dating
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.4,8,6.5,100,Paid,$12.99,Everyone,Medical
1231,Sway Medical,MEDICAL,5.0,3,22.0,100,Free,0,Everyone,Medical
1247,Chat Kids - Chat Room For Kids,DATING,4.7,6,4.9,100,Free,0,Mature 17+,Dating


In [25]:
# 중복된 데이터 App 컬럼을 기준으로 필터링 - Instagram
df_apps_clean[df_apps_clean.App == "Instagram"]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.5,66577446,5.3,"1,000,000,000",Free,0,Teen,Social
10809,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.5,66509917,5.3,"1,000,000,000",Free,0,Teen,Social


In [27]:
# 중복된 데이터 제거 - 주의, 중복된 데이터 제거시 중복 항목을 식별할 수 있는 방법을 명시해야 함(컬럼 명 명시)
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'], ignore_index=True)
df_apps_clean

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
0,KBA-EZ Health Guide,MEDICAL,5.0,4,25.00,1,Free,0,Everyone,Medical
1,Ra Ga Ba,GAME,5.0,2,20.00,1,Paid,$1.49,Everyone,Arcade
2,Mu.F.O.,GAME,5.0,2,16.00,1,Paid,$0.99,Everyone,Arcade
3,Brick Breaker BR,GAME,5.0,7,19.00,5,Free,0,Everyone,Arcade
4,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.0,1,4.60,5,Free,0,Everyone,Medical
...,...,...,...,...,...,...,...,...,...,...
8194,Google Drive,PRODUCTIVITY,4.4,2731171,4.00,"1,000,000,000",Free,0,Everyone,Productivity
8195,YouTube,VIDEO_PLAYERS,4.3,25655305,4.65,"1,000,000,000",Free,0,Teen,Video Players & Editors
8196,Google Play Movies & TV,VIDEO_PLAYERS,3.7,906384,4.65,"1,000,000,000",Free,0,Teen,Video Players & Editors
8197,Google News,NEWS_AND_MAGAZINES,3.9,877635,13.00,"1,000,000,000",Free,0,Teen,News & Magazines


In [28]:
# 중복 제거 후 필터링한 "Instagram" 확인
df_apps_clean[df_apps_clean.App == "Instagram"]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
8188,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social


- 현재 1차 전처리된 데이터에는 중복 항목이 있는 것이 발견되었다. 중복된 데이터의 개수는 모두 476개였다. (".duplicated()" 함수 사용)
- App 컬럼에 "Instagram"을 필터링 해보면 "Instagram"의 데이터가 여러개 있는 것을 알 수 있다. 
- 중복값 제거를 위해 ".drop_duplicates()" 함수를 사용할 것이다. 여기서 주의해야 할 점은 중복 항목을 식별할 수 있는 방법을 명시하지 않으면, 중복 제거가 제대로 되지 않는다. 
- 중복 항목을 식별하기 위해, 비교해야 할 열 이름을 제공해준다. (.drop_duplicates()안에 매개변수 "subset"을 이용하여 컬럼을 지정해주는데, 여기서는 'App', 'Type', 'Price' 컬럼이다.)
- 그리고 중복값이 제거되면 해당 행의 데이터는 제거가 되어 제거된 행의 index가 비게된다. 이때 "ignore_index=True"라고 설정해주면, index를 재배열 해준다.
- 제거 완료 후 데이터의 개수는 "8,199개의 행", "10개의 컬럼"으로 전처리 되었다. 

#### 1-3. 데이터에 대해 알아야 할 추가사항
- 이제 13개의 다른 기능들이 구글 플레이 스토어에서 스크랩되었다는 것을 알게 되었다. 
- 명백하게도, 이 데이터들은 모든 안드로이드 앱 중 샘플에 불과하다(수백만 개의 안드로이드 앱을 모두 포함하고 있지는 않는다).
- 샘플은 앱 스토어 전체를 대표하는 샘플이라고 가정한다. 웹 스크랩 처리 중에, 해당 샘플은 스크랩한 사람의 지리적인 위치(여기서는 Lavanya Gupta 위치의 사용자 사용)와 사용자의 행동에 의해 제공되었기 때문에 이게 전부다 그런 것은 아니다.
- 데이터는 2017/2018년 경에 작성되었으며, 금액 데이터는 스크래핑 당시의 가격을 USD로 반영한 것이다. (개발자는 프로모션을 제공할 수 있고, 앱의 가격을 변경할 수 있다)
- 앱의 크기는 MB 단위의 부동 소수점 형태로 변환했다. 데이터가 없다면, 해당 범주의 평균 크기로 대체했다.
- 설치 횟수는 정확하지 않다. 245,239회의 설치 횟수를 보였다면 구글은 이를 단순히 100,000회 이상으로 보고할 것이다. + 기호는 제거했으며, 단순화를 위해 해당 열에 정확한 설치 수를 가정하였다.
- 위 내용들은 구글플레이스토어의 목록으로 이동할 때 안드로이드 앱 목록에 표시되는 내용이다. 아래의 이미지를 참고하자.

<img src = "data/sample.png">